In [1]:
from neuralnilm.source import RealApplianceSource, MultiSource, SameLocation
import matplotlib.pyplot as plt
import numpy as np

/home/jack/workspace/python/nntools/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


In [2]:
MAX_TARGET_POWER = 300

real_appliance_source1 = RealApplianceSource(
    filename='/data/mine/vadeec/merged/ukdale.h5',
    appliances=[
        ['fridge freezer', 'fridge', 'freezer'],
        ['washer dryer', 'washing machine'],
        'hair straighteners',
        'television',
        'dish washer'
    ],
    max_appliance_powers=[MAX_TARGET_POWER, 2400, 500, 200, 2500],
    on_power_thresholds=[5] * 5,
    min_on_durations=[60, 1800, 60, 60, 1800],
    min_off_durations=[12, 600, 12, 12, 1800],
    divide_input_by_max_input_power=False,
    window=("2013-03-18", "2013-04-18"),
    seq_length=512,
    output_one_appliance=True,
    train_buildings=[1],
    validation_buildings=[1],
    n_seq_per_batch=8,
    skip_probability=0.75,
    skip_probability_for_first_appliance=0.5,
    target_is_start_and_end_and_mean=True,
    standardise_input=True
)

Loading training activations...


INFO:neuralnilm.source:Loading training activations...


  Loading activations for fridge freezer from building 1...


INFO:neuralnilm.source:  Loading activations for fridge freezer from building 1...


Loaded 760 activations.


...

INFO:neuralnilm.source:  Loading activations for dish washer from building 1...


Loaded 11 activations.


INFO:neuralnilm.source:Loaded 11 activations.


Done loading activations.


INFO:neuralnilm.source:Done loading activations.


In [4]:
same_location_source1 = SameLocation(
    filename='/data/mine/vadeec/merged/ukdale.h5',
    target_appliance='fridge freezer',
    window=("2013-04-18", "2013-05-18"),
    seq_length=512,
    train_buildings=[1],
    validation_buildings=[1],
    n_seq_per_batch=8,
    skip_probability=0.75,
    target_is_start_and_end_and_mean=True,
    standardise_input=True,
    offset_probability=1,
    divide_target_by=MAX_TARGET_POWER
)

Loaded 751 fridge freezer activations from house 1.


INFO:neuralnilm.source:Loaded 751 fridge freezer activations from house 1.


Loaded mains data for building 1.


INFO:neuralnilm.source:Loaded mains data for building 1.


In [5]:
multi_source = MultiSource(
    sources=[
        {
            'source': real_appliance_source1,
            'train_probability': 0.5,
            'validation_probability': 0
        },
        {
            'source': same_location_source1,
            'train_probability': 0.5,
            'validation_probability': 1
        }
    ],
    standardisation_source=same_location_source1
)

In [32]:
batch = multi_source.get_batch()#
#same_location_source1.skip_probability=0
# batch = same_location_source1.get_batch()
#batch = real_appliance_source1.get_batch()
X, y = batch.data
print(X.shape)
print(y.shape)
print(y[0, :, 0])
fig, axes = plt.subplots(2, sharex=True)
axes[0].plot(X[0, :, 0])
axes[1].plot(batch.target_power_timeseries[0, :, 0])
plt.show()

Using source_i 1
(8, 512, 1)
(8, 3, 1)
[ 0.06445312  0.27148438  0.28647798]


In [ ]:
batch = same_location_source1.get_batch()